In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

import scipy.stats as sts
import bpe

import keras as K
import keras.layers as L
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import ModelCheckpoint
from keras.models import load_model

from IPython.display import HTML, display_html
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Загружаем подготовленные данные

In [2]:
features = pickle.load(open('./processed/rnn_features.pkl', 'rb'))
labels = pd.read_csv('./processed/labels.csv')

bpe_encoder = pickle.load(open('./processed/pbe_encoder.pkl', 'rb'))

### Делим дданные на трейн для обучения и отложенную тестовую выборку

In [3]:
x_train, x_test, y_train, y_test = train_test_split(features, labels.values, test_size=0.2)

### Функция построения модели

Модель должна быть легкой и быстрой, поэтому поставлю себе ограничение на миллион параметров и возможность применять модель для $20-30$ текстов в секунду.


Обучать модель будем адамом с дефолтными параметрами, лосс - котегориальная кросэнтропия. Для этой функции потерь преопразуем предсказываемый вектор из размерности $shape=(7, )$, в вектор $shape=(7, 2)$ где позиция $[i, j], j \in \{0, 1\}$ таргета будет вероятностью $i$'ой позиции таргета быть $j$

Более детальное описание модели в отчете. 

In [4]:
def build_model():
    l_input = L.Input(shape=(None, ))
    l_in2 = L.Embedding(input_dim=bpe_encoder.vocab_size, output_dim=10)(l_input)
    
    # (batch, len, 10) -> (batch, len, 128)
    l_in3 = L.TimeDistributed(L.Dense(units=128))(l_in2)
    
    # (batch, len, 128) -> (batch, len, 256)
    l_rnn1 = L.Bidirectional(L.LSTM(units=128, return_sequences=True))(l_in3)
    # (batch, len, 256) -> (batch, len, 256)
    l_rnn2 = L.Bidirectional(L.LSTM(units=128, return_sequences=True))(l_rnn1)
    
    # (batch, len, 256) -> (batch, len, 128)
    l_dense1 = L.TimeDistributed(L.Dense(units=128))(l_rnn2)
    # (batch, len, 128) -> (batch, 128)
    l_comb = L.GlobalMaxPool1D()(l_dense1)
    # (batch, 128) -> (batch, 128)
    l_dence2 = L.Dense(units=128, activation='relu')(l_comb)
    # (batch, 128) -> (batch, 14)
    l_final = L.Dense(units=2 * 7)(l_dence2)
    # (batch, 14) -> (batch, 7, 2)
    l_final_reshape = L.Reshape(target_shape=(7, 2))(l_final)
    # (batch, 7, 2) -> (batch, 7, 2)
    l_prob = L.Softmax(axis=2)(l_final_reshape)
    
    model = K.Model(input=l_input, output=l_prob)
    model.compile(optimizer='adam', loss='categorical_crossentropy',
    )
    return model

Используйте одну из следующих двух ячеек чтобы сделать новую модель, или загрузить модель из файла.

*В репозитории на github должна лежать самая хорошая модель*

In [5]:
model = build_model()

W0929 22:17:03.279319 140483696359232 deprecation_wrapper.py:119] From /home/michael/.virtualenv/DS3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0929 22:17:03.314570 140483696359232 deprecation_wrapper.py:119] From /home/michael/.virtualenv/DS3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0929 22:17:03.318742 140483696359232 deprecation_wrapper.py:119] From /home/michael/.virtualenv/DS3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/home/michael/.virtualenv/DS3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("so...)`
W0929 22:17:04.463596 14

In [10]:
model = load_model('./models/model4_30.hdf5')

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_20 (Embedding)     (None, None, 10)          81920     
_________________________________________________________________
time_distributed_32 (TimeDis (None, None, 128)         1408      
_________________________________________________________________
bidirectional_33 (Bidirectio (None, None, 256)         263168    
_________________________________________________________________
bidirectional_34 (Bidirectio (None, None, 256)         394240    
_________________________________________________________________
time_distributed_33 (TimeDis (None, None, 128)         32896     
_________________________________________________________________
global_max_pooling1d_15 (Glo (None, 128)               0         
__________

Получилась довольно компактная модель

Напишем функцию - генератор батчей для обучения

In [12]:
def generator_batch1(X, Y, batch_size=6, smooth=0.2, unk_prob=0.07):
    
    # сохраним значение токенов Паддинга (pad) и Неизвестного токена (unk)
    UNK_num = list(bpe_encoder.transform([bpe_encoder.UNK]))[0][0]
    PAD_num = list(bpe_encoder.transform([bpe_encoder.PAD]))[0][0]
    
    print(f'unk: {UNK_num}, pad: {PAD_num}')
    
    # Отсортируем наши данные по количеству токенов в тектсте
    buf = sorted([(len(x), x, y) for (x, y) in zip(X, Y) ], key=lambda x: x[0])
    X, Y = zip(*[(x, y) for _, x, y in buf ])
    X = np.array(X)
    Y = np.array(Y)
    
    while True:
        # далее сохраним возможные индексы начала батча и отсортируем их
        # батч будем формировать беря очередной индекс и следующие batch_size
        #   элементов за ним
        # таким образом мы получим случайную последовательность проходящую по всем
        #   элементам выборки, и в добавок каждый батч будет содержать примерно равные
        #   по длинне последовательности токенов 
        indexes = np.arange(len(X) - batch_size)
        np.random.shuffle(indexes)
        
        for ind in indexes:
            # делаем one-hot encoding таргета
            y = to_categorical(np.expand_dims(Y[ind : ind + batch_size, :], axis=2), 2)
            
            # применяем смуфинг, этот прием помогает делать сеть не такой уверенной в своих
            #   выводах, что хорошо влияет на метрики на отложенной выборке
            y[y == 1] = 1 - smooth
            y[y == 0] = smooth
            
            # дополняем все последовательности текущего батча до макимальной длинны,
            #   заполняя короткие последовательности Паддингами
            x = pad_sequences(X[ind : ind + batch_size], padding='post', value=PAD_num)
            
            # еще один небольшой хак, давайте с некоторой вероятностью заменять реальные
            #   токены на Неизвестный токен - unk, интуитивное объяснение этому, в том что
            #   сеть начинает делать предсказания в условии меньшей информации и меньше
            #   полагается на конкретные слова, так как они могут 'выпасть' 
            len_min = len(X[ind])
            x_unk = np.random.binomial(1, unk_prob, (batch_size, len_min))
            x[:, :len_min][x_unk == 1] = UNK_num
            
            # и наконец отправим батч с сетку
            yield (x, y)

Будем сохранять промежуточныйе модели по ходу обучения

In [13]:
checkpointer = ModelCheckpoint(
    filepath=r"./models/model_{epoch:02d}.hdf5", 
    save_best_only=False,
    save_weights_only=False,
    period=2
)

реально это клеточка запусклась 4 раза, так что при запуске на только что инициализированной модели, значения лосса будут чуть выше, примерно 0.58 в среднем за эпоху

In [279]:
model.fit_generator(
    generator=generator_batch1(x_train, y_train),
    steps_per_epoch=500,
    epochs=30,
    callbacks=[checkpointer]
)

Epoch 1/30
unk: 1, pad: 0
500/500 [==============================] - 97s 194ms/step - loss: 0.5159
Epoch 2/30
500/500 [==============================] - 102s 203ms/step - loss: 0.5157
Epoch 3/30
500/500 [==============================] - 97s 194ms/step - loss: 0.5168
Epoch 4/30
500/500 [==============================] - 112s 224ms/step - loss: 0.5143
Epoch 5/30
500/500 [==============================] - 105s 209ms/step - loss: 0.5158
Epoch 6/30
500/500 [==============================] - 99s 198ms/step - loss: 0.5140
Epoch 7/30
500/500 [==============================] - 93s 186ms/step - loss: 0.5138
Epoch 8/30
500/500 [==============================] - 110s 221ms/step - loss: 0.5147
Epoch 9/30
500/500 [==============================] - 97s 194ms/step - loss: 0.5145
Epoch 10/30
500/500 [==============================] - 116s 232ms/step - loss: 0.5150
Epoch 11/30
500/500 [==============================] - 111s 223ms/step - loss: 0.5153
Epoch 12/30
500/500 [==============================] 

#### Посчитаем метрики модели

Данные очень несбалансированные, $\sim 89 \%$ данный вообще не содержат ни одной метки. Поэтому метрика точности (accuracy), которую я считаю, почти ни о чем не говорит. Так что я счиатаю roc auc, но и accuracy тоже, просто она мне нравится.

In [15]:
%%time

predictions = np.array(list(map(
    lambda x: np.argmax(model.predict(np.array([x]))[0], axis=1),
    x_test[:5000],
)))

CPU times: user 4min 23s, sys: 16.4 s, total: 4min 39s
Wall time: 1min 25s


In [16]:
labels_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] + ['toxic_content']

for i in range(7):
    print(labels_column[i])
    print('accuracy', end=': ')
    print(accuracy_score(predictions[:, i], y_test[:5000, i]))
    print('rocauc', end=': ')
    print(roc_auc_score(y_test[:5000, i], predictions[:, i]))
    print()

toxic
accuracy: 0.9686
rocauc: 0.8721912068591522

severe_toxic
accuracy: 0.9906
rocauc: 0.7255538192848607

obscene
accuracy: 0.9846
rocauc: 0.914971079033579

threat
accuracy: 0.9974
rocauc: 0.5

insult
accuracy: 0.973
rocauc: 0.8274095715911618

identity_hate
accuracy: 0.9904
rocauc: 0.5

toxic_content
accuracy: 0.969
rocauc: 0.8832260068914872



предсказания по всему

In [17]:
roc_auc_score(y_test[:5000], predictions)

0.7461930976657486

без искуственно добавленной метки

In [18]:
roc_auc_score(y_test[:5000, :6], predictions[:, :6])

0.7233542794614589

## Немного поанализируем модель


Попытка сделать бонусное задание по интропретируемости.

Посмотрим, на что модельно обращает внимание больше, а на что меньше. Для этого есть инересный подход, заменять последовательно слова на UNK и смотреть, как меняется оцнека моделью вероятностей.

In [19]:
# всмомогательная функция рисования цветных слов
# ПИСАЛ НЕ САМ! Взято из второго семинара курса ШАДа по NLP читавшегося осенью 2018 года.

def draw_html(tokens_and_weights, cmap=plt.get_cmap("bwr"), display=True,
              token_template="""<span style="background-color: {color_hex}">{token}</span>""",
              font_style="font-size:14px;"
             ):
    
    def get_color_hex(weight):
        rgba = cmap(1. / (1 + np.exp(weight)), bytes=True)
        return '#%02X%02X%02X' % rgba[:3]
    
    tokens_html = [
        token_template.format(token=token, color_hex=get_color_hex(weight))
        for token, weight in tokens_and_weights
    ]
    
    
    raw_html = """<p style="{}">{}</p>""".format(font_style, ' '.join(tokens_html))
    if display:
        display_html(HTML(raw_html))
        
    return raw_html

Теперь сам напишу функцию, которая будет считать вклады слов в итоговый скор.

Функция принимает модель, токены и функцю извлечения нужной метрики из предсказания.

In [20]:
def get_wieght(model, tokens, weight_extracter):
    # посчитает базовый скор, без унков
    base_score = weight_extracter(model.predict(np.array([tokens]))[0])
    
    # сохраняем Токен унка
    UNK_num = list(bpe_encoder.transform([bpe_encoder.UNK]))[0][0]
    
    # веса
    weights = []
    position = 0
    
    while position < len(tokens):
        token = tokens[position]
        
        # если слово
        if token in bpe_encoder.inverse_bpe_vocab.keys():
            infer = process_tokenized_word(model, tokens, position, base_score, weight_extracter, UNK_num)
        
        # если нечто токенизированное
        if token in bpe_encoder.inverse_word_vocab.keys():
            infer = process_single_word(model, tokens, position, base_score, weight_extracter, UNK_num)
            
        weight, word, positino_up = infer
        
        weights.append((word, weight))
        position += positino_up
        
    return weights, base_score

def process_single_word(model, tokens, position, base_score, weight_extracter, UNK):
    buf = tokens.copy()
    buf[position] = UNK

    # считаем вклад токена
    predictions = model.predict(np.array([buf]))[0]
    weight = weight_extracter(predictions) - base_score
    
    return (weight, bpe_encoder.inverse_word_vocab[tokens[position]], 1)

def process_tokenized_word(model, tokens, position, base_score, weight_extracter, UNK):
    # считаем вклад токенезированного слова как сумму вкладов его частей
    sm = 0
    for i in range(position, len(tokens)):
        buf = tokens.copy()
        buf[i] = UNK
        
        # считаем вклад токена
        predictions = model.predict(np.array([buf]))[0]
        weight = weight_extracter(predictions)
        sm += weight - base_score
        
        # если токен был концом слова, вернем вес, само слово и новую позицию
        if tokens[i] == bpe_encoder.bpe_vocab['__eow']:
            word = next(bpe_encoder.inverse_transform([tokens[position : i + 1]]))
            return (sm, word, i)

### Понаслаждаемся, раскрасим некоторые предложения по влиянию на токсичность


К сожалению цветовые теги не отображаются в просмоторщике github

In [21]:
def drow_tokcik(tokens, realy=None):
    tokens_and_weights, base_score = get_wieght(model, tokens, lambda x: x[0][1])
    print(f'It was tixic for { str(round(base_score, 4))[:4] }')
    if realy is not None:
        print(f'And realy {realy[0]}')
    draw_html([(tok, weight * 100) for tok, weight in tokens_and_weights]);

In [22]:
drow_tokcik(x_test[1], y_test[1])

It was tixic for 0.20
And realy 0


please undo your second reversion of my removal of that taxobox gn is

In [35]:
drow_tokcik(x_test[180], y_test[180])

It was tixic for 0.22
And realy 0


fun !!! lol jk ts

In [32]:
drow_tokcik(x_test[12], y_test[12])

It was tixic for 0.2
And realy 0


, you don ' t call respected review sites numrd party coverage ?

In [25]:
drow_tokcik(x_test[186], y_test[186])

It was tixic for 0.19
And realy 0


posting styles . . that is woefully ie nt context here .

In [47]:
drow_tokcik(x_test[34], y_test[34])

It was tixic for 0.21
And realy 0


acs are either another blackjack di sa bi li ty the other hand are part of a community of anonymous creeps

In [50]:
drow_tokcik(x_test[157], y_test[157])

It was tixic for 0.20
And realy 0


tu an article here that the budget dispute is going to court now . i didnt read the specifics though . that is really cool you got that photo ! any word on when the new city hall / old verizon stuck it ... i seem to have been distracted

Иногда модель обращает внимание на какой-то бред :(

### Эксперементы быстродействия

In [440]:
train_data = pd.read_csv('./data/train.csv')
texts = list(train_data.comment_text)

In [447]:
def get_single_prediction(text_line, model):
    tokens = next(bpe_encoder.transform([text_line]))
    prediction = np.argmax(model.predict(np.array([tokens]))[0], axis=1)

In [450]:
%%time

list(map(lambda x : get_single_prediction(x, model), texts[:100 * 1000]))

CPU times: user 1h 34min 11s, sys: 5min 52s, total: 1h 40min 3s
Wall time: 31min 3s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Это заняло $94$ минуты процессорного времени. То есть $57ms$ на полный процессинг одного текста. Что кажется довольно быстро, учитывая возможность процессить батчами.

*Во время всего обучения и процессингов использовались CPU: 4 ядра Intel Core5*

In [452]:
(94 * 60 + 11) / 100 / 1000

0.05651